## Introduction


[Idefics2](https://huggingface.co/HuggingFaceM4/idefics2-8b) is a powerful vision language model from Huggingface that has been pre-trained and fine-tuned to take arbitrary text and image input and generate text output, with improved performance on visual question and answering and OCR over the first generation. It is released under Apache 2.0, along with its multimodal instruction fine-tuning dataset [The Cauldron](https://huggingface.co/datasets/HuggingFaceM4/the_cauldron).

In this notebook, I am going to fine-tune the model for multi-page contract document QA. For a while, document QA can only been done with single-page documents, while in reality, one could have 100+ pages in a single agreement. With that being said, it is still difficult for this model to support beyond 10 pages with a GPU-poor man's budget. Some features that are supported by Idefics2, such as sub-image tokenization (64 * 5 = 320 tokens for one page instead of 64) and high resolution (980 * 980) are also disabled for my experiment due to the hardware constraints. 

## Dataset

### Load the Raw Data

I have been collecting public contract dataset for a while. `chenghao/sec-material-contracts` is an ongoing project that crawls contract documents from the [EDGAR](sec.gov) website. It includes both the raw html contract content and all relevant metadata.

In [8]:
import datetime
import time
import io
from urllib.parse import urljoin

import fitz
from tqdm.notebook import tqdm

from bs4 import BeautifulSoup
from datasets import load_dataset
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from trafilatura import html2txt
from weasyprint import HTML
from PIL import Image

In [ ]:
source_repo = "chenghao/sec-material-contracts"
dataset = load_dataset(source_repo)

### Convert HTML to PDF and Images

We want the model to be able to read the document images and answer the query accordingly. This means we need to convert the HTML code into PDF images. To do that, there is some html clean-up I need to take care first: converting relative urls to absolute urls to preserve any image data; 

This is optional depending on how you convert them into PDFs or how much image matters to the downstream application. In this case, it is cheap enough so I am adding it here. 


In [9]:
def convert_relative_url_to_absolute_url(base_url, html):
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup.find_all("a", href=True):
        tag["href"] = urljoin(base_url, tag["href"])
    for tag in soup.find_all("img", src=True):
        tag["src"] = urljoin(base_url, tag["src"])
    return str(soup)

In [10]:
def convert_html_to_pdf(html):
    doc = fitz.Document(stream=io.BytesIO(HTML(string=html).write_pdf()), filetype="pdf")
    images = [page.get_pixmap(dpi=120) for page in doc]
    images = [Image.frombytes("RGB", (image.width, image.height), image.samples) for image in images]
    page_text = [page.get_text() for page in doc]

    return page_text, images

Now we are ready to do the actual conversion and extraction. Namely, I am extracting the following columns:
1. Full text from the HTML document, which is used later for GPT-4o labelling;
2. PDF image pages;
3. Page text;
4. Cleaned up HTML content for reproducibility;

In [11]:
def extract_text(record):
    file_content = record["file_content"]

    if not file_content:
        return {"full_text": "", "images": [], "page_text": [], "html_content": ""}

    file_content_lower = file_content.lower()
    if "<html>" not in file_content_lower or "</html>" not in file_content_lower:
        return {"full_text": "", "images": [], "page_text": [], "html_content": ""}

    left = file_content_lower.index("<html>")
    right = file_content_lower.index("</html>")
    html_content = file_content[left + len("<html>") : right]

    full_text = html2txt(html_content)
    index_url = record["index_html_url"]
    base_url = index_url.replace("-index.html", "").replace("-", "") + "/"
    html_content = convert_relative_url_to_absolute_url(base_url, html_content)
    page_text, images = convert_html_to_pdf(html_content)

    return {
        "full_text": full_text,
        "images": images,
        "page_text": page_text,
        "html_content": html_content,
    }

### Sampling

There are more than 800k documents in the current data repo. For the fine-tuning purpose, I am going to take sample documents that are uploaded this year so that:
1. Documents are less likely to be scanned or of less quality assuming the natural progression and development of modern technology;
2. Language used in those documents reflects more closely with current writing or contractual style;

Some additional filtering rules are also applied here:
1. Non empty text;
2. Maximum 20 page of text;
3. Minimum 50 characters in each page;

In [ ]:
cutoff = datetime.datetime(2024, 1, 1)
sample = dataset["train"].filter(lambda x: x["date"] >= cutoff)
sample = sample.map(lambda x: extract_text(x), num_proc=8)

In [ ]:
final_sample = sample.filter(
    lambda x: x["full_text"]
    and 1 <= len(x["page_text"]) <= 20
    and min(map(len, x["page_text"])) >= 50,
    num_proc=8,
)

### Key Information Extraction

To compose a QA dataset, I need to come up with question and answer pairs somehow. Luckily, commercial large models are excellent at information extraction at this level, especially when you have access to the full text.

You might ask: if you have the full text, isn't it easier to build a model directly on such data instead of modelling with images? It is true to some extent that text-based model can handle some of the QA tasks pretty well. But documents with a complex layout can pose a challenge that few text extraction or OCR tools can handle reasonably well.

To create the dataset, I am going to ask GPT-4o to read the entire document full text and extract the key information with some help from langchain to enforce structured output. 

In [ ]:
# https://support.ironcladapp.com/hc/en-us/articles/12947738534935-Ironclad-AI-Overview
class KeyInformation(BaseModel):
    agreement_date: str = Field(
        description="Agreement signing date of the contract. (date)"
    )
    effective_date: str = Field(description="Effective date of the contract. (date)")
    expiration_date: str = Field(
        description="Service end date or expiration date of the contract. (date)"
    )
    party_address: str = Field(description="Address of the party to the contract.")
    party_name: str = Field(description="The names of the contracting party.")
    counterparty_address: str = Field(
        description="Address of the counterparty to the contract."
    )
    counterparty_name: str = Field(
        description="The names of the contracting counterparty."
    )
    counterparty_signer_name: str = Field(
        description="The name of the counterparty signer for each party to the agreement."
    )
    counterparty_signer_title: str = Field(
        description="The counterparty signer’s title (e.g., CEO)."
    )
    auto_renewal: str = Field(
        description="Whether the contract term automatically renews (true/false)."
    )
    governing_law: str = Field(description="(Jurisdiction) Choice of law.")
    venue: str = Field(
        description="Location of the courts where legal proceedings will take place."
    )
    payment_frequency: str = Field(
        description="The cadence for which payments are made (e.g., monthly, annually, one-time)."
    )
    payment_term: str = Field(
        description="When an invoice is due after issuance (e.g. Net 30)"
    )
    renewal_term: str = Field(
        description="The length of time the renewal period will last (e.g., 1 year, 2 years, 24 months etc.)."
    )
    agreement_term: str = Field(
        description="Term of the contract as an amount of time (e.g., 24 months)."
    )
    termination_for_cause: str = Field(
        description="Whether one or all parties may terminate the contract with cause, such as a breach of contract (true/false)."
    )
    termination_for_convenience: str = Field(
        description="Whether one or all parties may terminate the contract without cause, or at their convenience (true/false)."
    )
    termination_notice_period: str = Field(
        description="The period by which notice of termination must be given (e.g., 30 days)."
    )
    opt_out_length: str = Field(
        description="Required notice period to NOT renew (e.g., 30 days)."
    )
    contract_value: str = Field(
        description="Total fixed fee amount including currency codes or symbols. (monetary amount)"
    )


load_dotenv()
model = ChatOpenAI(model="gpt-4o", max_retries=2).with_structured_output(KeyInformation)
chain = model

In [ ]:
def text2qa(record):
    page_text = record["page_text"]
    content = "\n\n".join(page_text)

    messages = [
        SystemMessage(
            content="""You are a legal expert who is helping a client understand a contract. The client asks you to extract the key information for the given contract and return them in a structured format. Use N/A if not applicable or not available."""
        ),
        HumanMessage(content=content),
    ]

    results = chain.invoke(messages)

    return dict(results)

Since it is relatively expensive to call the model, I am going to use a simple loop to retry calling the endpoint if it failed for some reason.

In [ ]:
data = final_sample
results = [None for _ in data]
errors = list(range(len(data)))
while errors:
    new_errors = []
    for i in tqdm.tqdm(errors):
        try:
            qa = text2qa(data[i])
            results[i] = qa
            time.sleep(5)

        except Exception as e:
            print(f"Error at {i}: {e}")
            new_errors.append(i)
            continue

    errors = new_errors
    cont = input(f"Found {len(errors)} errors. Continue? (y/n)")
    if cont.lower() != "y":
        break

### Saving the dataset

The datasets is saved both locally and on Hugging Face Hub for future use.

In [ ]:
import pandas as pd
from datasets import Dataset

output = []
for i, record in enumerate(final_sample):
    output.append(record | results[i])

df = pd.DataFrame(output)
ds = Dataset.from_pandas(df)
ds.save_to_disk("temp-data")
ds.push_to_hub("chenghao/sec-material-contracts-qa")

## Model Training

For this tutorial, I am going to use Lightning + transformers/peft to train the model. First, let me define some helper functions:

In [ ]:
# pip install datasets peft bitsandbytes accelerate wandb "git+https://github.com/huggingface/transformers" autoawq lightning nltk loguru
import os
import re

import lightning as L
import numpy as np
import torch
from loguru import logger
from datasets import concatenate_datasets
from datasets import load_dataset, load_from_disk
from lightning.pytorch.callbacks import Callback
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import WandbLogger
from nltk import edit_distance
from peft import LoraConfig
from peft import prepare_model_for_kbit_training, get_peft_model
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from transformers import AutoProcessor
from transformers import BitsAndBytesConfig, Idefics2ForConditionalGeneration
try:
    from deepspeed.ops.adam import FusedAdam
except ImportError:
    from torch.optim import Adam as FusedAdam

A helper function to convert the dataset to the QA format that Idefics2 can understand.

In [ ]:
def convert_key_info_to_qa(records: dict) -> dict:
    """
    Convert key information into QA format.

    Parameters
    ----------
    records : dict
        The records from the dataset.

    Returns
    -------
    dict
        The QA format of the records.
    """
    key2question = {
        "agreement_date": "When is the signing date of this agreement?",
        "effective_date": "When is the effective date of the contract?",
        "expiration_date": "When is the service end date or expiration date of the contract?",
        "party_address": "What is the address of the party to the contract?",
        "party_name": "What are the names of the contracting party?",
        "counterparty_address": "What is the address of the counterparty to the contract?",
        "counterparty_name": "What are the names of the contracting counterparty?",
        "counterparty_signer_name": "What is the name of the counterparty signer for each party to the agreement?",
        "counterparty_signer_title": "What is the counterparty signer’s title?",
        "auto_renewal": "Whether the contract term automatically renews (true/false).",
        "governing_law": "Where is the jurisdiction or choice of law?",
        "venue": "where is the location of the courts where legal proceedings will take place?",
        "payment_frequency": "what is the cadence for which payments are made (e.g., monthly, annually, one-time)?",
        "payment_term": "When an invoice is due after issuance (e.g. Net 30)?",
        "renewal_term": "What is the length of time the renewal period will last (e.g., 1 year, 2 years, 24 months etc.)?",
        "agreement_term": "What is the term of the contract as an amount of time (e.g., 24 months)?",
        "termination_for_cause": "Whether one or all parties may terminate the contract with cause, such as a breach of contract (true/false).",
        "termination_for_convenience": "Whether one or all parties may terminate the contract without cause, or at their convenience (true/false).",
        "termination_notice_period": "What is the period by which notice of termination must be given (e.g., 30 days)?",
        "opt_out_length": "What is the required notice period to NOT renew (e.g., 30 days)?",
        "contract_value": "What is the total fixed fee amount including currency codes or symbols?",
    }
    images = records["images"][0]
    questions, answers = [], []
    for key in key2question:
        answer = records[key][0]
        if answer != "N/A":
            questions.append(key2question[key])
            answers.append(answer)

    output = {
        "images": [images for _ in questions],
        "answer": answers,
        "question": questions
    }
    return output

Create a custom dataset for Idefics2 training. Specifically, we can specify a maximum page number for each document so that we can control the token length for each document. Here, I am using both the head and tail of the document and skipping the middle part. This is based on the assumption that the middle part is the most template-like part of the document.

In [ ]:
class Idefics2Dataset(Dataset):
    """
    PyTorch Dataset for Idefics2. This class takes a HuggingFace Dataset as input.
    """

    def __init__(self, ds, max_page):
        super().__init__()
        self.dataset = ds
        self.max_page = max_page
        assert max_page > 0

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, idx: int) -> tuple:
        sample = self.dataset[idx]

        if len(sample["images"]) > self.max_page:
            images = sample["images"][:self.max_page // 2] + sample["images"][-self.max_page // 2:]
        else:
            images = sample["images"]

        question = sample["question"]
        answer = sample["answer"]

        return images, question, answer


For a custom dataset, it is important to create custom collate function to handle the mini batching. The idea is to format each example into the training format by concatenating the images and the question, pad the input according to the maximum length, and then return the input ids, attention mask, pixel values, and pixel attention mask.

In [ ]:
def train_collate_fn(examples):
    global processor
    global image_token_id

    texts = []
    images = []
    for example in examples:
        images_example, question, answer = example
        content = [{"type": "image"} for _ in range(len(images_example))]
        content += [{"type": "text", "text": question}]

        # Create inputs
        messages = [
            {
                "role": "user",
                "content": content,
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": answer},
                ]
            },
        ]

        prompt = processor.apply_chat_template(messages, add_generation_prompt=False)
        texts.append(prompt)
        images.append(images_example)

    batch = processor(text=texts, images=images, padding=True, truncation=True, max_length=MAX_LENGTH,
                      return_tensors="pt")

    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    batch["labels"] = labels

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    pixel_values = batch["pixel_values"]
    pixel_attention_mask = batch["pixel_attention_mask"]
    labels = batch["labels"].long()

    return input_ids, attention_mask, pixel_values, pixel_attention_mask, labels


def eval_collate_fn(examples):
    images = []
    texts = []
    answers = []
    for example in examples:
        images_example, question, answer = example

        content = [{"type": "image"} for _ in range(len(images_example))]
        content += [{"type": "text", "text": question}]

        messages = [
            {
                "role": "user",
                "content": content,
            },
        ]
        text = processor.apply_chat_template(messages, add_generation_prompt=True)
        images.append(images_example)
        texts.append(text.strip())
        answers.append(answer)

    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    pixel_values = batch["pixel_values"]
    pixel_attention_mask = batch["pixel_attention_mask"]

    return input_ids, attention_mask, pixel_values, pixel_attention_mask, answers


### Model Definition

Along with the model definition, I also included the callback from the original tutorial so that model can be pushed to the hub after training.

In [ ]:
class Idefics2ModelPLModule(L.LightningModule):
    def __init__(self, model_config, inp_processor, torch_model):
        super().__init__()
        self.config = model_config
        self.processor = inp_processor
        self.model = torch_model
        self.batch_size = self.config.get("batch_size")

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, pixel_values, pixel_attention_mask, labels = batch
        outputs = self.model(input_ids=input_ids,
                             attention_mask=attention_mask,
                             pixel_values=pixel_values,
                             pixel_attention_mask=pixel_attention_mask,
                             labels=labels
                             )
        loss = outputs.loss
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx, dataset_idx=0):
        global MAX_LENGTH

        input_ids, attention_mask, pixel_values, pixel_attention_mask, answers = batch

        generated_ids = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            pixel_values=pixel_values,
            pixel_attention_mask=pixel_attention_mask,
            max_new_tokens=MAX_LENGTH
        )
        # turn them back into text, chopping of the prompt
        # important: we don't skip special tokens here, because we want to see them in the output
        predictions = self.processor.batch_decode(generated_ids[:, input_ids.size(1):], skip_special_tokens=True)

        scores = []
        for pred, answer in zip(predictions, answers):
            pred = re.sub(r"((?<=>) | (?=</s_))", "", pred)
            scores.append(edit_distance(pred, answer) / max(len(pred), len(answer)))

            if self.config.get("verbose", False) and len(scores) == 1:
                logger.debug(f"""
Prediction: `{pred}`
Answer: `{answer}`
Normed ED: `{scores[0]:.3f}`""")

        self.log("val_edit_distance", np.mean(scores))

        return scores

    def configure_optimizers(self):
        # you could also add a learning rate scheduler if you want
        optimizer = FusedAdam(self.parameters(), lr=self.config.get("lr"))

        return optimizer

    def train_dataloader(self):
        global train_dataset
        return DataLoader(train_dataset, collate_fn=train_collate_fn, batch_size=self.batch_size, shuffle=True,
                          num_workers=6)

    def val_dataloader(self):
        global validation_dataset
        return DataLoader(validation_dataset, collate_fn=eval_collate_fn, batch_size=self.batch_size, shuffle=False,
                          num_workers=6)


class PushToHubCallback(Callback):

    def on_train_epoch_end(self, pl_trainer, pl_module):
        global FINETUNED_REPO_ID
        logger.info(f"Pushing model to the hub, epoch {pl_trainer.current_epoch}")
        pl_module.model.push_to_hub(
            FINETUNED_REPO_ID,
            commit_message=f"Training in progress, epoch {pl_trainer.current_epoch}"
        )

    def on_train_end(self, pl_trainer, pl_module):
        global FINETUNED_REPO_ID
        logger.info("Pushing model to the hub after training")
        pl_module.processor.push_to_hub(FINETUNED_REPO_ID, commit_message="Training done")
        pl_module.model.push_to_hub(FINETUNED_REPO_ID, commit_message="Training done")


### Training

To start training, I am using the following configuration:

1. QLoRA
2. Maximum page size of 5
3. Batch size of 2 with gradient accumulation of 12
4. Learning rate of 1e-4
5. Number of epochs of 10
6. Gradient clipping of 1.0
7. Adam optimizer (or FusedAdam) 
8. Maximum length of 1024 tokens
9. Warmup steps of 50
10. No image splitting so only 64 tokens per image
11. Reduced image resolution to maximum of 350 * 490 pixels

In [ ]:
MAX_LENGTH = 1024
USE_LORA = False
USE_QLORA = True
MAX_PAGE = 5
FINETUNED_REPO_ID = "chenghao/idefics2-edgar"
WANDB_PROJECT = "Idefics2-EDGAR"
WANDB_NAME = "demo-run"
config = {
    "max_epochs": 10,
    # "val_check_interval": 0.2,
    "check_val_every_n_epoch": 1,
    "gradient_clip_val": 1.0,
    "accumulate_grad_batches": 12,
    "lr": 1e-4,
    "batch_size": 2,
    "precision": "16-mixed",
    "seed": 42,
    "warmup_steps": 50,
    "result_path": "./result",
    "verbose": True,
}

wandb_logger = WandbLogger(project=WANDB_PROJECT, name=WANDB_NAME)
processor = AutoProcessor.from_pretrained(
    "HuggingFaceM4/idefics2-8b",
    do_image_splitting=False,
    size={"longest_edge": 490, "shortest_edge": 350}
)
image_token_id = processor.tokenizer.additional_special_tokens_ids[processor.tokenizer.additional_special_tokens.index("<image>")]

Split the dataset into train and validation set if not done before.

In [ ]:
if os.path.exists("local-dataset"):
    dataset = load_from_disk("local-dataset")
else:
    dude_dataset = load_dataset("jordyvl/DUDE_subset_100val")
    edgar_dataset = load_dataset("chenghao/sec-material-contracts-qa")
    flattened_edgar_dataset = edgar_dataset['train'].map(
        convert_key_info_to_qa, batched=True, batch_size=1,
        remove_columns=edgar_dataset['train'].column_names, num_proc=10)
    # flattened_edgar_dataset = flattened_edgar_dataset.filter(lambda x: len(x['images']) <= MAX_PAGE, num_proc=10)
    dude_dataset = dude_dataset.remove_columns(["questionId"])
    flattened_edgar_dataset = flattened_edgar_dataset.cast(dude_dataset['train'].features)
    all_dataset = concatenate_datasets([dude_dataset['train'], flattened_edgar_dataset])
    dataset = all_dataset.train_test_split(test_size=0.2)
    dataset.save_to_disk("local-dataset")

train_dataset = Idefics2Dataset(dataset["train"], max_page=MAX_PAGE)
validation_dataset = Idefics2Dataset(dataset["test"], max_page=MAX_PAGE)

Load the model according to the configurations.

QLoRA essentially means quantization + LoRA, which is a technique that allows you to fine-tune a pre-trained model on a dataset with low-precision data types, such as 8-bit or 4-bit quantization and low-rank adaptation (LoRA) techniques. This technique can significantly reduce the memory footprint of the model and improve its performance for training on low-precision hardware.

Here, I am using 4-bit NormalFloat (NF4), a new data type that is information theoretically optimal for normally distributed weights, and double quantization to reduce the average memory footprint by quantizing the quantization constants. For the LoRA part, I am using rank 8 and a dropout rate of 0.1.

In [ ]:
if USE_QLORA or USE_LORA:
    if USE_QLORA:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16
        )
    else:
        quantization_config = None

    model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        quantization_config=quantization_config,
    )
else:
    # for full fine-tuning, we can speed up the model using Flash Attention
    # only available on certain devices, see
    # https://github.com/Dao-AILab/flash-attention?tab=readme-ov-file#installation-and-features
    model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        _attn_implementation="flash_attention_2",
    )

if USE_QLORA or USE_LORA:
    lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=".*(text_model|modality_projection|perceiver_resampler).*(down_proj|gate_proj|up_proj|k_proj|q_proj|v_proj|o_proj).*$",
        init_lora_weights="gaussian",
    )

    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)

In [ ]:
model_module = Idefics2ModelPLModule(config, processor, model)

early_stop_callback = EarlyStopping(monitor="val_edit_distance", patience=3, verbose=False, mode="min")

trainer = L.Trainer(
    accelerator="auto",
    devices=1,
    strategy="deepspeed_stage_2",
    max_epochs=config.get("max_epochs"),
    check_val_every_n_epoch=config.get("check_val_every_n_epoch"),
    gradient_clip_val=config.get("gradient_clip_val"),
    accumulate_grad_batches=config.get("accumulate_grad_batches"),
    precision=config.get("precision"),
    num_sanity_val_steps=10,
    logger=wandb_logger,
    callbacks=[PushToHubCallback(), early_stop_callback],
)

trainer.fit(model_module)

## Evaluation

Next, I am going to load the model from the hub and evaluate it on the validation set.

In [ ]:
import torch
from transformers import AutoProcessor, Idefics2ForConditionalGeneration, BitsAndBytesConfig

base_model = "HuggingFaceM4/idefics2-8b"
peft_model_id = "chenghao/idefics2-edgar"
# peft_model_id = "HuggingFaceM4/idefics2-8b"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = Idefics2ForConditionalGeneration.from_pretrained(
    peft_model_id,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

model.eval()

processor = AutoProcessor.from_pretrained(
    base_model,
    do_image_splitting=False,
    size={"longest_edge": 490, "shortest_edge": 350}
)

In [ ]:
from tqdm.auto import tqdm

def evaluate():
    
    questions = []
    answers = []
    predictions = []
    
    for example in tqdm(ds["test"]):
        images = example["images"][:2] + example["images"][-2:]
        question, answer = example["question"], example["answer"]
        
        messages = [
            {
                "role": "user",
                "content": [{"type": "image"} for _ in range(len(images))] + [{"type": "text", "text": question}],
            },
        ]

        prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
        inputs = processor(text=prompt, images=images, return_tensors="pt").to("cuda")
        with torch.no_grad():
            generated_ids = model.generate(**inputs, max_new_tokens=1024)
        generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
        preds = [t.split("Assistant:", 1)[-1].strip() for t in generated_texts]

        questions.append(question)
        answers.append(answer)
        predictions.append(preds[0])
    
    return questions, answers, predictions

questions, answers, predictions = evaluate()
# ! Save them somewhere for later

For the evaluation, I am using the edit distance to measure the similarity between the predicted answer and the ground truth answer. The edit distance is calculated as the number of insertions, deletions, and substitutions required to transform the predicted answer into the ground truth answer. The edit distance is then normalized by the length of the longest answer and the shortest answer to get a score between 0 and 1. 

The mean of all the edit distance scores is used as the evaluation metric (either by label (macro) or by question (micro)).

In [ ]:
from nltk import edit_distance
import numpy as np
from collections import defaultdict

key2question = {
    "agreement_date": "When is the signing date of this agreement?",
    "effective_date": "When is the effective date of the contract?",
    "expiration_date": "When is the service end date or expiration date of the contract?",
    "party_address": "What is the address of the party to the contract?",
    "party_name": "What are the names of the contracting party?",
    "counterparty_address": "What is the address of the counterparty to the contract?",
    "counterparty_name": "What are the names of the contracting counterparty?",
    "counterparty_signer_name": "What is the name of the counterparty signer for each party to the agreement?",
    "counterparty_signer_title": "What is the counterparty signer’s title?",
    "auto_renewal": "Whether the contract term automatically renews (true/false).",
    "governing_law": "Where is the jurisdiction or choice of law?",
    "venue": "where is the location of the courts where legal proceedings will take place?",
    "payment_frequency": "what is the cadence for which payments are made (e.g., monthly, annually, one-time)?",
    "payment_term": "When an invoice is due after issuance (e.g. Net 30)?",
    "renewal_term": "What is the length of time the renewal period will last (e.g., 1 year, 2 years, 24 months etc.)?",
    "agreement_term": "What is the term of the contract as an amount of time (e.g., 24 months)?",
    "termination_for_cause": "Whether one or all parties may terminate the contract with cause, such as a breach of contract (true/false).",
    "termination_for_convenience": "Whether one or all parties may terminate the contract without cause, or at their convenience (true/false).",
    "termination_notice_period": "What is the period by which notice of termination must be given (e.g., 30 days)?",
    "opt_out_length": "What is the required notice period to NOT renew (e.g., 30 days)?",
    "contract_value": "What is the total fixed fee amount including currency codes or symbols?",
}
question2key = {q: k for k, q in key2question.items()}

def calculate_edit_distance(questions, answers, predictions):
    scores = defaultdict(list)
    for question, pred, answer in tqdm(zip(questions, predictions, answers), total=len(answers)):
        if question not in question2key:
            continue
        question = question2key[question]
        scores[question].append(edit_distance(pred, answer) / max(len(pred), len(answer)))
    return {question: np.mean(values) for question, values in scores.items()}, np.mean([v for values in scores.values() for v in values])

Plotting the results with a bar chart with the results saved from the previous cell.

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import json

def calculate_df(file1, file2):
    
    with open(file1) as f:
        d1 = json.load(f)
        r1, a1 = calculate_edit_distance(d1['questions'], d1['answers'], d1['predictions'])
    
    
    with open(file2) as f:
        d2 = json.load(f)
        r2, a2 = calculate_edit_distance(d2['questions'], d2['answers'], d2['predictions'])
    
    print(f"{(a1-a2)/a1*100:.2f}%")
    records = []
    name = {
        "base": "Idefics2-8B",
        "finetuned": "Idefics2-8B-EDGAR"
    }
    for key, value in r1.items():
        n = name[file1.replace(".json", "")]
        records.append((key, value, n))
    for key, value in r2.items():
        n = name[file2.replace(".json", "")]
        records.append((key, value, n))
    
    return pd.DataFrame(records, columns=["category", "value", "model"])


sns.set_theme(font_scale=1.4)
fig, ax = plt.subplots(figsize=(20, 20))
df = calculate_df("base.json", "finetuned.json")
sns.barplot(y=df.category, x=df.value, hue=df.model, ax=ax)
ax.set_xlabel("Average Edit Distance (lower is better)")
ax.set_ylabel("Category")

ax.set_title("Comparison between base and finetuned model")

A more detailed breakdown of the differences between the two models for each category can be found in the following table:

In [ ]:
data = []
for category, row in df.groupby("category"):
    base = row[row["model"] == "Idefics2-8B"]["value"].iloc[0]
    after = row[row["model"] != "Idefics2-8B"]["value"].iloc[0]
    delta = (base - after) / base
    data.append((category, base, after, f"{delta * 100:.2f}%"))

print(pd.DataFrame(data, columns=["Category", "Idefics2-8B", "Idefics2-8B-EDGAR", "Δ(↑)"]).to_markdown())